# 02 - Data Processing

In [1]:
import sys
sys.executable

'/usr/local/bin/python'

## Imports

In [2]:
import numpy as np
import pandas as pd

import constants as cnst
import stock_utils as su

pd.set_option('display.max_columns', None)

NSE_DATA_DIR = PosixPath('../data/NSE') | Valid: True
PROCESSED_DATA_DIR = PosixPath('../data/processed') | Valid: True


## Constants

In [3]:
stock_symbols = su.get_all_stock_symbols(
    cnst.NSE_DATA_DIR
)

stock_symbols

['DEEPAKFERT', 'HDFCBANK', 'INDIGOPNTS', 'ITBEES', 'JUBLFOOD']

In [4]:
STOCK_SYMBOL = stock_symbols[2]
STOCK_SYMBOL

'INDIGOPNTS'

## Data loading

In [5]:
stock_df = pd.read_parquet(cnst.NSE_DATA_DIR.joinpath(STOCK_SYMBOL, "consolidated.parquet"))
stock_df = stock_df.drop(columns = ['series', 'PREV. CLOSE'])
stock_df.columns = [
    'Date',
    'Open', 'High', 'Low', 'LTP', 'Close',
    'VWAP', '52W H', '52W L',
    'Volume', 'Value', '#Trades'
]
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades
0,2021-02-02,2607.50,3129.00,2436.05,3129.00,3117.15,2684.35,3129.00,2436.05,12652036,3.396245e+10,812580
1,2021-02-03,3239.00,3329.95,2831.25,2953.45,2924.25,3085.39,3329.95,2436.05,2955005,9.117329e+09,375333
2,2021-02-04,2948.85,2967.95,2853.00,2866.00,2873.75,2902.67,3329.95,2436.05,476054,1.381829e+09,73464
3,2021-02-05,2866.00,2900.00,2603.00,2621.00,2630.05,2694.77,3329.95,2436.05,587315,1.582680e+09,103675
4,2021-02-08,2590.00,2736.00,2551.00,2700.00,2677.05,2666.44,3329.95,2436.05,507665,1.353659e+09,55033
...,...,...,...,...,...,...,...,...,...,...,...,...
755,2024-02-19,1517.00,1517.00,1495.10,1512.85,1511.80,1507.80,1679.95,981.95,59787,9.014660e+07,7776
756,2024-02-20,1521.00,1521.00,1492.25,1503.00,1503.50,1503.27,1679.95,981.95,48951,7.358659e+07,6424
757,2024-02-21,1505.00,1515.95,1475.50,1490.10,1492.05,1496.00,1679.95,981.95,66167,9.898576e+07,9274
758,2024-02-22,1487.00,1488.90,1446.05,1451.00,1451.05,1460.34,1679.95,981.95,98408,1.437093e+08,14285


In [6]:
stock_df.info(memory_usage = "deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   Date     760 non-null    datetime64[ns]
 1   Open     760 non-null    float64       
 2   High     760 non-null    float64       
 3   Low      760 non-null    float64       
 4   LTP      760 non-null    float64       
 5   Close    760 non-null    float64       
 6   VWAP     760 non-null    float64       
 7   52W H    760 non-null    float64       
 8   52W L    760 non-null    float64       
 9   Volume   760 non-null    int64         
 10  Value    760 non-null    float64       
 11  #Trades  760 non-null    int64         
dtypes: datetime64[ns](1), float64(9), int64(2)
memory usage: 71.4 KB


## Feature engineering

### Daily candle based columns

In [7]:
stock_df['Range'] = stock_df['High'] - stock_df['Low']
stock_df['IsGreen'] = (stock_df['Close'] >= stock_df['Open']).astype(int)
stock_df['Is52WLow'] = np.isclose(stock_df['Low'], stock_df['52W L'], atol = 0, rtol = 1e-4).astype(int)
stock_df['Is52WHigh'] = np.isclose(stock_df['High'], stock_df['52W H'], atol = 0, rtol = 1e-4).astype(int)
stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh
142,2021-08-31,2575.00,2612.05,2554.00,2565.15,2585.95,2586.68,3329.95,2220.25,33767,8.734438e+07,5575,58.05,1,0,0
303,2022-04-26,1571.00,1590.00,1546.75,1575.10,1575.25,1573.47,2740.00,1438.15,19895,3.130423e+07,3518,43.25,1,0,0
107,2021-07-09,2550.00,2559.95,2515.55,2520.00,2518.95,2527.91,3329.95,2220.25,14658,3.705415e+07,2700,44.40,0,0,0
376,2022-08-08,1500.25,1559.95,1500.25,1530.00,1528.45,1532.86,2695.00,1353.60,51537,7.899884e+07,7351,59.70,1,0,0
718,2023-12-27,1468.55,1504.45,1464.20,1497.05,1498.40,1483.54,1679.95,981.95,96798,1.436034e+08,13082,40.25,1,0,0
232,2022-01-10,2070.00,2098.60,2053.50,2074.95,2075.15,2074.64,3329.95,1900.00,56544,1.173086e+08,4899,45.10,1,0,0
10,2021-02-16,2600.00,2617.10,2557.00,2561.00,2564.55,2576.40,3329.95,2436.05,68502,1.764883e+08,10257,60.10,0,0,0
596,2023-06-30,1425.00,1449.00,1425.00,1429.95,1428.90,1432.44,1731.95,981.95,56911,8.152159e+07,6092,24.00,1,0,0
160,2021-09-27,2600.00,2608.95,2515.00,2533.00,2529.40,2547.52,3329.95,2220.25,30511,7.772732e+07,5980,93.95,0,0,0
291,2022-04-06,1613.00,1643.95,1600.75,1612.90,1609.60,1621.65,2740.00,1438.15,30889,5.009103e+07,3724,43.20,0,0,0


In [8]:
((stock_df[['IsGreen', 'Is52WLow', 'Is52WHigh']].sum() * 100) / len(stock_df)).round(2)

IsGreen      32.89
Is52WLow      8.03
Is52WHigh     0.26
dtype: float64

In [9]:
stock_df.shape

(760, 16)

### Date based columns

In [10]:
stock_df['Day'] = stock_df['Date'].dt.day
stock_df['Month'] = stock_df['Date'].dt.month
stock_df['Year'] = stock_df['Date'].dt.year
stock_df['Weekday'] = stock_df['Date'].dt.weekday
stock_df['DayOfYear'] = stock_df['Date'].dt.day_of_year
stock_df['Quarter'] = stock_df['Date'].dt.quarter
stock_df['DaysSinceLastTradingSession'] = (stock_df['Date'] - stock_df['Date'].shift(1)).dt.days.fillna(1).astype(int)

stock_df.sample(10)

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession
221,2021-12-24,1960.00,1967.90,1905.05,1930.0,1914.15,1918.90,3329.95,1905.05,25649,4.921777e+07,5155,62.85,0,1,0,24,12,2021,4,358,4,1
350,2022-07-01,1404.50,1405.00,1372.70,1388.0,1384.15,1386.31,2709.00,1353.60,12205,1.691993e+07,3247,32.30,0,0,0,1,7,2022,4,182,3,1
191,2021-11-11,2349.90,2357.10,2323.00,2332.0,2326.30,2333.09,3329.95,2220.25,7939,1.852236e+07,1976,34.10,0,0,0,11,11,2021,3,315,4,1
121,2021-07-30,2567.00,2625.00,2560.00,2600.0,2605.10,2595.82,3329.95,2220.25,23850,6.191021e+07,5195,65.00,1,0,0,30,7,2021,4,211,3,1
608,2023-07-18,1506.00,1529.70,1471.70,1502.0,1491.60,1502.44,1731.95,981.95,233607,3.509812e+08,20985,58.00,0,0,0,18,7,2023,1,199,3,1
13,2021-02-19,2636.00,2660.00,2563.70,2575.0,2572.60,2605.85,3329.95,2436.05,66948,1.744565e+08,10360,96.30,0,0,0,19,2,2021,4,50,1,1
484,2023-01-13,1271.90,1289.55,1269.25,1284.0,1281.70,1279.39,2108.00,1240.00,16106,2.060592e+07,2863,20.30,1,0,0,13,1,2023,4,13,1,1
437,2022-11-09,1350.25,1471.70,1331.00,1430.0,1430.95,1389.13,2395.00,1331.00,1414851,1.965407e+09,47412,140.70,1,1,0,9,11,2022,2,313,4,2
34,2021-03-23,2390.00,2395.00,2334.05,2367.0,2367.25,2364.54,3329.95,2220.25,40074,9.475648e+07,5846,60.95,0,0,0,23,3,2021,1,82,1,1
612,2023-07-24,1501.25,1512.30,1486.00,1500.0,1493.15,1497.77,1731.95,981.95,61937,9.276734e+07,7615,26.30,0,0,0,24,7,2023,0,205,3,3


In [11]:
stock_df.shape

(760, 23)

### Moving averages

#### `Close`

In [12]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Close {window}MA'] = stock_df['Close'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Close.*")

,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA
0,3117.15,3117.15,3117.15,3117.15,3117.15
1,2924.25,3020.70,3020.70,3020.70,3020.70
2,2873.75,2971.72,2971.72,2971.72,2971.72
3,2630.05,2886.30,2886.30,2886.30,2886.30
4,2677.05,2844.45,2844.45,2844.45,2844.45
...,...,...,...,...,...
755,1511.80,1473.09,1449.51,1460.62,1472.00
756,1503.50,1484.54,1453.20,1461.24,1472.52
757,1492.05,1490.64,1456.44,1461.30,1472.68
758,1451.05,1487.30,1458.40,1460.31,1471.87


#### `Range`

In [13]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Range {window}MA'] = stock_df['Range'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "Range.*")

,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA
0,692.95,692.95,692.95,692.95,692.95
1,498.70,595.82,595.82,595.82,595.82
2,114.95,435.53,435.53,435.53,435.53
3,297.00,400.90,400.90,400.90,400.90
4,185.00,357.72,357.72,357.72,357.72
...,...,...,...,...,...
755,21.90,55.49,47.82,45.23,36.15
756,28.75,43.88,46.74,45.16,36.13
757,40.45,40.30,47.90,45.71,36.48
758,42.85,39.04,47.95,46.40,36.79


#### `VWAP`

In [14]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'VWAP {window}MA'] = stock_df['VWAP'].rolling(window = window, min_periods = 1).mean().round(2)

stock_df.filter(regex = "VWAP.*")

,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA
0,2684.35,2684.35,2684.35,2684.35,2684.35
1,3085.39,2884.87,2884.87,2884.87,2884.87
2,2902.67,2890.80,2890.80,2890.80,2890.80
3,2694.77,2841.80,2841.80,2841.80,2841.80
4,2666.44,2806.72,2806.72,2806.72,2806.72
...,...,...,...,...,...
755,1507.80,1482.80,1455.74,1463.98,1474.14
756,1503.27,1490.49,1459.14,1464.43,1474.61
757,1496.00,1493.76,1462.89,1464.66,1474.92
758,1460.34,1491.52,1464.93,1464.15,1474.39


#### `Volume`

In [15]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Volume {window}MA'] = stock_df['Volume'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Volume.*")

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,12652036,12652036,12652036,12652036,12652036
1,2955005,7803520,7803520,7803520,7803520
2,476054,5361031,5361031,5361031,5361031
3,587315,4167602,4167602,4167602,4167602
4,507665,3435615,3435615,3435615,3435615
...,...,...,...,...,...
755,59787,229740,141834,134431,98886
756,48951,170199,142950,134887,98819
757,66167,104563,144848,135534,99479
758,98408,94564,148361,137455,100396


#### `Value`

In [16]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'Value {window}MA'] = stock_df['Value'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "Value.*")

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,3.396245e+10,33962446981,33962446981,33962446981,33962446981
1,9.117329e+09,21539888009,21539888009,21539888009,21539888009
2,1.381829e+09,14820534938,14820534938,14820534938,14820534938
3,1.582680e+09,11511071310,11511071310,11511071310,11511071310
4,1.353659e+09,9479588872,9479588872,9479588872,9479588872
...,...,...,...,...,...
755,9.014660e+07,338786942,207849552,197851208,145956748
756,7.358659e+07,252862455,209635951,198552770,145880554
757,9.898576e+07,156387430,212617675,199532276,146875939
758,1.437093e+08,141408559,217840828,202316250,148192613


#### `#Trades`

In [17]:
for window in cnst.ROLLING_WINDOWS:
    stock_df[f'#Trades {window}MA'] = stock_df['#Trades'].rolling(window = window, min_periods = 1).mean().astype(int)

stock_df.filter(regex = "#Trades.*")

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,812580,812580,812580,812580,812580
1,375333,593956,593956,593956,593956
2,73464,420459,420459,420459,420459
3,103675,341263,341263,341263,341263
4,55033,284017,284017,284017,284017
...,...,...,...,...,...
755,7776,24460,16612,13663,10986
756,6424,18176,16698,13675,10962
757,9274,12797,16863,13722,11043
758,14285,11945,17202,13916,11152


### Target columns

In [18]:
for window in cnst.TARGET_WINDOWS:
    stock_df[f'Target {window}D'] = stock_df['Close'].shift(-window)

stock_df.filter(regex = "Target.*")

,Target 3D,Target 7D,Target 15D,Target 30D
0,2630.05,2649.40,2510.95,2367.80
1,2677.05,2610.75,2487.95,2305.80
2,2704.85,2595.65,2499.70,2285.45
3,2666.25,2564.55,2522.20,2378.50
4,2649.40,2642.50,2506.15,2367.25
...,...,...,...,...
755,1451.05,NaN,NaN,NaN
756,1443.50,NaN,NaN,NaN
757,NaN,NaN,NaN,NaN
758,NaN,NaN,NaN,NaN


### Saving processed data

In [19]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-processed.parquet'), 
    index = False
)
stock_df

,Date,Open,High,Low,LTP,Close,VWAP,52W H,52W L,Volume,Value,#Trades,Range,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,2021-02-02,2607.50,3129.00,2436.05,3129.00,3117.15,2684.35,3129.00,2436.05,12652036,3.396245e+10,812580,692.95,1,1,1,2,2,2021,1,33,1,1,3117.15,3117.15,3117.15,3117.15,692.95,692.95,692.95,692.95,2684.35,2684.35,2684.35,2684.35,12652036,12652036,12652036,12652036,33962446981,33962446981,33962446981,33962446981,812580,812580,812580,812580,2630.05,2649.40,2510.95,2367.80
1,2021-02-03,3239.00,3329.95,2831.25,2953.45,2924.25,3085.39,3329.95,2436.05,2955005,9.117329e+09,375333,498.70,0,0,1,3,2,2021,2,34,1,1,3020.70,3020.70,3020.70,3020.70,595.82,595.82,595.82,595.82,2884.87,2884.87,2884.87,2884.87,7803520,7803520,7803520,7803520,21539888009,21539888009,21539888009,21539888009,593956,593956,593956,593956,2677.05,2610.75,2487.95,2305.80
2,2021-02-04,2948.85,2967.95,2853.00,2866.00,2873.75,2902.67,3329.95,2436.05,476054,1.381829e+09,73464,114.95,0,0,0,4,2,2021,3,35,1,1,2971.72,2971.72,2971.72,2971.72,435.53,435.53,435.53,435.53,2890.80,2890.80,2890.80,2890.80,5361031,5361031,5361031,5361031,14820534938,14820534938,14820534938,14820534938,420459,420459,420459,420459,2704.85,2595.65,2499.70,2285.45
3,2021-02-05,2866.00,2900.00,2603.00,2621.00,2630.05,2694.77,3329.95,2436.05,587315,1.582680e+09,103675,297.00,0,0,0,5,2,2021,4,36,1,1,2886.30,2886.30,2886.30,2886.30,400.90,400.90,400.90,400.90,2841.80,2841.80,2841.80,2841.80,4167602,4167602,4167602,4167602,11511071310,11511071310,11511071310,11511071310,341263,341263,341263,341263,2666.25,2564.55,2522.20,2378.50
4,2021-02-08,2590.00,2736.00,2551.00,2700.00,2677.05,2666.44,3329.95,2436.05,507665,1.353659e+09,55033,185.00,1,0,0,8,2,2021,0,39,1,3,2844.45,2844.45,2844.45,2844.45,357.72,357.72,357.72,357.72,2806.72,2806.72,2806.72,2806.72,3435615,3435615,3435615,3435615,9479588872,9479588872,9479588872,9479588872,284017,284017,284017,284017,2649.40,2642.50,2506.15,2367.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,2024-02-19,1517.00,1517.00,1495.10,1512.85,1511.80,1507.80,1679.95,981.95,59787,9.014660e+07,7776,21.90,0,0,0,19,2,2024,0,50,1,3,1473.09,1449.51,1460.62,1472.00,55.49,47.82,45.23,36.15,1482.80,1455.74,1463.98,1474.14,229740,141834,134431,98886,338786942,207849552,197851208,145956748,24460,16612,13663,10986,1451.05,NaN,NaN,NaN
756,2024-02-20,1521.00,1521.00,1492.25,1503.00,1503.50,1503.27,1679.95,981.95,48951,7.358659e+07,6424,28.75,0,0,0,20,2,2024,1,51,1,1,1484.54,1453.20,1461.24,1472.52,43.88,46.74,45.16,36.13,1490.49,1459.14,1464.43,1474.61,170199,142950,134887,98819,252862455,209635951,198552770,145880554,18176,16698,13675,10962,1443.50,NaN,NaN,NaN
757,2024-02-21,1505.00,1515.95,1475.50,1490.10,1492.05,1496.00,1679.95,981.95,66167,9.898576e+07,9274,40.45,0,0,0,21,2,2024,2,52,1,1,1490.64,1456.44,1461.30,1472.68,40.30,47.90,45.71,36.48,1493.76,1462.89,1464.66,1474.92,104563,144848,135534,99479,156387430,212617675,199532276,146875939,12797,16863,13722,11043,NaN,NaN,NaN,NaN
758,2024-02-22,1487.00,1488.90,1446.05,1451.00,1451.05,1460.34,1679.95,981.95,98408,1.437093e+08,14285,42.85,0,0,0,22,2,2024,3,53,1,1,1487.30,1458.40,1460.31,1471.87,39.04,47.95,46.40,36.79,1491.52,1464.93,1464.15,1474.39,94564,148361,137455,100396,141408559,217840828,202316250,148192613,11945,17202,13916,11152,NaN,NaN,NaN,NaN


## Data standardization

### Stock price based columns

In [20]:
stock_price_cols = [
    'Open', 'High', 'Low', 'LTP', '52W H', '52W L'
] + (
    stock_df.filter(regex = "Close.*").columns.to_list() +
    stock_df.filter(regex = "Range.*").columns.to_list() +
    stock_df.filter(regex = "VWAP.*").columns.to_list() +
    stock_df.filter(regex = "Target.*").columns.to_list()
)

stock_price_cols

['Open',
 'High',
 'Low',
 'LTP',
 '52W H',
 '52W L',
 'Close',
 'Close 7MA',
 'Close 15MA',
 'Close 30MA',
 'Close 60MA',
 'Range',
 'Range 7MA',
 'Range 15MA',
 'Range 30MA',
 'Range 60MA',
 'VWAP',
 'VWAP 7MA',
 'VWAP 15MA',
 'VWAP 30MA',
 'VWAP 60MA',
 'Target 3D',
 'Target 7D',
 'Target 15D',
 'Target 30D']

In [21]:
stock_df[stock_price_cols] = stock_df[stock_price_cols].div(stock_df['Close'], axis = 0).round(3)
stock_df[stock_price_cols]

,Open,High,Low,LTP,52W H,52W L,Close,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,1.004,0.781,1.0,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,0.861,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.139,0.833,1.0,1.033,1.033,1.033,1.033,0.171,0.204,0.204,0.204,0.204,1.055,0.987,0.987,0.987,0.987,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.159,0.848,1.0,1.034,1.034,1.034,1.034,0.040,0.152,0.152,0.152,0.152,1.010,1.006,1.006,1.006,1.006,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.266,0.926,1.0,1.097,1.097,1.097,1.097,0.113,0.152,0.152,0.152,0.152,1.025,1.081,1.081,1.081,1.081,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,1.244,0.910,1.0,1.063,1.063,1.063,1.063,0.069,0.134,0.134,0.134,0.134,0.996,1.048,1.048,1.048,1.048,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1.003,1.003,0.989,1.001,1.111,0.650,1.0,0.974,0.959,0.966,0.974,0.014,0.037,0.032,0.030,0.024,0.997,0.981,0.963,0.968,0.975,0.960,NaN,NaN,NaN
756,1.012,1.012,0.993,1.000,1.117,0.653,1.0,0.987,0.967,0.972,0.979,0.019,0.029,0.031,0.030,0.024,1.000,0.991,0.970,0.974,0.981,0.960,NaN,NaN,NaN
757,1.009,1.016,0.989,0.999,1.126,0.658,1.0,0.999,0.976,0.979,0.987,0.027,0.027,0.032,0.031,0.024,1.003,1.001,0.980,0.982,0.989,NaN,NaN,NaN,NaN
758,1.025,1.026,0.997,1.000,1.158,0.677,1.0,1.025,1.005,1.006,1.014,0.030,0.027,0.033,0.032,0.025,1.006,1.028,1.010,1.009,1.016,NaN,NaN,NaN,NaN


### Volume based columns

In [22]:
volume_cols = stock_df.filter(regex = "Volume.*").columns.to_list()

volume_cols

['Volume', 'Volume 7MA', 'Volume 15MA', 'Volume 30MA', 'Volume 60MA']

In [23]:
stock_df[volume_cols] = stock_df[volume_cols].div(stock_df['Volume'], axis = 0).round(3)
stock_df[volume_cols]

,Volume,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.641,2.641,2.641,2.641
2,1.0,11.261,11.261,11.261,11.261
3,1.0,7.096,7.096,7.096,7.096
4,1.0,6.767,6.767,6.767,6.767
...,...,...,...,...,...
755,1.0,3.843,2.372,2.248,1.654
756,1.0,3.477,2.920,2.756,2.019
757,1.0,1.580,2.189,2.048,1.503
758,1.0,0.961,1.508,1.397,1.020


### Value based columns

In [24]:
value_cols = stock_df.filter(regex = "Value.*").columns.to_list()

value_cols

['Value', 'Value 7MA', 'Value 15MA', 'Value 30MA', 'Value 60MA']

In [25]:
stock_df[value_cols] = stock_df[value_cols].div(stock_df['Value'], axis = 0).round(3)
stock_df[value_cols]

,Value,Value 7MA,Value 15MA,Value 30MA,Value 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,2.363,2.363,2.363,2.363
2,1.0,10.725,10.725,10.725,10.725
3,1.0,7.273,7.273,7.273,7.273
4,1.0,7.003,7.003,7.003,7.003
...,...,...,...,...,...
755,1.0,3.758,2.306,2.195,1.619
756,1.0,3.436,2.849,2.698,1.982
757,1.0,1.580,2.148,2.016,1.484
758,1.0,0.984,1.516,1.408,1.031


### Trade count based columns

In [26]:
trade_count_cols = stock_df.filter(regex = "#Trades.*").columns.to_list()

trade_count_cols

['#Trades', '#Trades 7MA', '#Trades 15MA', '#Trades 30MA', '#Trades 60MA']

In [27]:
stock_df[trade_count_cols] = stock_df[trade_count_cols].div(stock_df['#Trades'], axis = 0).round(3)
stock_df[trade_count_cols]

,#Trades,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA
0,1.0,1.000,1.000,1.000,1.000
1,1.0,1.582,1.582,1.582,1.582
2,1.0,5.723,5.723,5.723,5.723
3,1.0,3.292,3.292,3.292,3.292
4,1.0,5.161,5.161,5.161,5.161
...,...,...,...,...,...
755,1.0,3.146,2.136,1.757,1.413
756,1.0,2.829,2.599,2.129,1.706
757,1.0,1.380,1.818,1.480,1.191
758,1.0,0.836,1.204,0.974,0.781


### Dropping unnecessary columns

In [28]:
stock_df = stock_df.drop(columns = ['Date', 'Close', 'Range', 'Volume', 'Value', '#Trades'])
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,0.861,1.004,0.781,1,1,1,2,2,2021,1,33,1,1,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.055,1.139,0.833,0,0,1,3,2,2021,2,34,1,1,1.033,1.033,1.033,1.033,0.204,0.204,0.204,0.204,0.987,0.987,0.987,0.987,2.641,2.641,2.641,2.641,2.363,2.363,2.363,2.363,1.582,1.582,1.582,1.582,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.010,1.159,0.848,0,0,0,4,2,2021,3,35,1,1,1.034,1.034,1.034,1.034,0.152,0.152,0.152,0.152,1.006,1.006,1.006,1.006,11.261,11.261,11.261,11.261,10.725,10.725,10.725,10.725,5.723,5.723,5.723,5.723,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.025,1.266,0.926,0,0,0,5,2,2021,4,36,1,1,1.097,1.097,1.097,1.097,0.152,0.152,0.152,0.152,1.081,1.081,1.081,1.081,7.096,7.096,7.096,7.096,7.273,7.273,7.273,7.273,3.292,3.292,3.292,3.292,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,0.996,1.244,0.910,1,0,0,8,2,2021,0,39,1,3,1.063,1.063,1.063,1.063,0.134,0.134,0.134,0.134,1.048,1.048,1.048,1.048,6.767,6.767,6.767,6.767,7.003,7.003,7.003,7.003,5.161,5.161,5.161,5.161,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1.003,1.003,0.989,1.001,0.997,1.111,0.650,0,0,0,19,2,2024,0,50,1,3,0.974,0.959,0.966,0.974,0.037,0.032,0.030,0.024,0.981,0.963,0.968,0.975,3.843,2.372,2.248,1.654,3.758,2.306,2.195,1.619,3.146,2.136,1.757,1.413,0.960,NaN,NaN,NaN
756,1.012,1.012,0.993,1.000,1.000,1.117,0.653,0,0,0,20,2,2024,1,51,1,1,0.987,0.967,0.972,0.979,0.029,0.031,0.030,0.024,0.991,0.970,0.974,0.981,3.477,2.920,2.756,2.019,3.436,2.849,2.698,1.982,2.829,2.599,2.129,1.706,0.960,NaN,NaN,NaN
757,1.009,1.016,0.989,0.999,1.003,1.126,0.658,0,0,0,21,2,2024,2,52,1,1,0.999,0.976,0.979,0.987,0.027,0.032,0.031,0.024,1.001,0.980,0.982,0.989,1.580,2.189,2.048,1.503,1.580,2.148,2.016,1.484,1.380,1.818,1.480,1.191,NaN,NaN,NaN,NaN
758,1.025,1.026,0.997,1.000,1.006,1.158,0.677,0,0,0,22,2,2024,3,53,1,1,1.025,1.005,1.006,1.014,0.027,0.033,0.032,0.025,1.028,1.010,1.009,1.016,0.961,1.508,1.397,1.020,0.984,1.516,1.408,1.031,0.836,1.204,0.974,0.781,NaN,NaN,NaN,NaN


### Saving standardized data

In [29]:
stock_df.to_parquet(
    cnst.PROCESSED_DATA_DIR.joinpath(f'{STOCK_SYMBOL}-standardized.parquet'), 
    index = False
)
stock_df

,Open,High,Low,LTP,VWAP,52W H,52W L,IsGreen,Is52WLow,Is52WHigh,Day,Month,Year,Weekday,DayOfYear,Quarter,DaysSinceLastTradingSession,Close 7MA,Close 15MA,Close 30MA,Close 60MA,Range 7MA,Range 15MA,Range 30MA,Range 60MA,VWAP 7MA,VWAP 15MA,VWAP 30MA,VWAP 60MA,Volume 7MA,Volume 15MA,Volume 30MA,Volume 60MA,Value 7MA,Value 15MA,Value 30MA,Value 60MA,#Trades 7MA,#Trades 15MA,#Trades 30MA,#Trades 60MA,Target 3D,Target 7D,Target 15D,Target 30D
0,0.837,1.004,0.781,1.004,0.861,1.004,0.781,1,1,1,2,2,2021,1,33,1,1,1.000,1.000,1.000,1.000,0.222,0.222,0.222,0.222,0.861,0.861,0.861,0.861,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,1.000,0.844,0.850,0.806,0.760
1,1.108,1.139,0.968,1.010,1.055,1.139,0.833,0,0,1,3,2,2021,2,34,1,1,1.033,1.033,1.033,1.033,0.204,0.204,0.204,0.204,0.987,0.987,0.987,0.987,2.641,2.641,2.641,2.641,2.363,2.363,2.363,2.363,1.582,1.582,1.582,1.582,0.915,0.893,0.851,0.789
2,1.026,1.033,0.993,0.997,1.010,1.159,0.848,0,0,0,4,2,2021,3,35,1,1,1.034,1.034,1.034,1.034,0.152,0.152,0.152,0.152,1.006,1.006,1.006,1.006,11.261,11.261,11.261,11.261,10.725,10.725,10.725,10.725,5.723,5.723,5.723,5.723,0.941,0.903,0.870,0.795
3,1.090,1.103,0.990,0.997,1.025,1.266,0.926,0,0,0,5,2,2021,4,36,1,1,1.097,1.097,1.097,1.097,0.152,0.152,0.152,0.152,1.081,1.081,1.081,1.081,7.096,7.096,7.096,7.096,7.273,7.273,7.273,7.273,3.292,3.292,3.292,3.292,1.014,0.975,0.959,0.904
4,0.967,1.022,0.953,1.009,0.996,1.244,0.910,1,0,0,8,2,2021,0,39,1,3,1.063,1.063,1.063,1.063,0.134,0.134,0.134,0.134,1.048,1.048,1.048,1.048,6.767,6.767,6.767,6.767,7.003,7.003,7.003,7.003,5.161,5.161,5.161,5.161,0.990,0.987,0.936,0.884
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
755,1.003,1.003,0.989,1.001,0.997,1.111,0.650,0,0,0,19,2,2024,0,50,1,3,0.974,0.959,0.966,0.974,0.037,0.032,0.030,0.024,0.981,0.963,0.968,0.975,3.843,2.372,2.248,1.654,3.758,2.306,2.195,1.619,3.146,2.136,1.757,1.413,0.960,NaN,NaN,NaN
756,1.012,1.012,0.993,1.000,1.000,1.117,0.653,0,0,0,20,2,2024,1,51,1,1,0.987,0.967,0.972,0.979,0.029,0.031,0.030,0.024,0.991,0.970,0.974,0.981,3.477,2.920,2.756,2.019,3.436,2.849,2.698,1.982,2.829,2.599,2.129,1.706,0.960,NaN,NaN,NaN
757,1.009,1.016,0.989,0.999,1.003,1.126,0.658,0,0,0,21,2,2024,2,52,1,1,0.999,0.976,0.979,0.987,0.027,0.032,0.031,0.024,1.001,0.980,0.982,0.989,1.580,2.189,2.048,1.503,1.580,2.148,2.016,1.484,1.380,1.818,1.480,1.191,NaN,NaN,NaN,NaN
758,1.025,1.026,0.997,1.000,1.006,1.158,0.677,0,0,0,22,2,2024,3,53,1,1,1.025,1.005,1.006,1.014,0.027,0.033,0.032,0.025,1.028,1.010,1.009,1.016,0.961,1.508,1.397,1.020,0.984,1.516,1.408,1.031,0.836,1.204,0.974,0.781,NaN,NaN,NaN,NaN
